<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/feature%2Fadd-torch-multi-task-learning/multi_task_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Get Data

In [1]:
!rm run_ner.py tasks.py utils_ner.py

rm: cannot remove 'run_ner.py': No such file or directory
rm: cannot remove 'tasks.py': No such file or directory
rm: cannot remove 'utils_ner.py': No such file or directory


In [2]:
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-torch-multi-task-learning/run_ner.py
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-torch-multi-task-learning/tasks.py
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-torch-multi-task-learning/utils_ner.py

run_ner.py          100%[===================>]  16.54K  --.-KB/s    in 0.001s  
tasks.py            100%[===================>]   5.38K  --.-KB/s    in 0s      
utils_ner.py        100%[===================>]  26.05K  --.-KB/s    in 0.002s  


In [3]:
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/train.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/test.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/dev.txt
!mkdir data_twitter && mv train.txt data_twitter && mv test.txt data_twitter && mv dev.txt data_twitter

train.txt           100%[===================>]   2.20M  --.-KB/s    in 0.1s    
test.txt            100%[===================>] 108.10K  --.-KB/s    in 0.02s   
dev.txt             100%[===================>] 160.83K  --.-KB/s    in 0.02s   


In [4]:
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-torch-multi-task-learning/ner_data/peyma/dev.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-torch-multi-task-learning/ner_data/peyma/train.txt
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-torch-multi-task-learning/ner_data/peyma/test.txt
!mkdir data_peyma && mv train.txt data_peyma && mv test.txt data_peyma && mv dev.txt data_peyma

dev.txt             100%[===================>] 297.21K  --.-KB/s    in 0.03s   
train.txt           100%[===================>]   2.64M  --.-KB/s    in 0.1s    
test.txt            100%[===================>] 359.39K  --.-KB/s    in 0.03s   


### Install Dependencies

In [5]:
!pip -q install transformers
!pip -q install sentencepiece
!pip -q install seqeval
!pip -q install conllu

     |████████████████████████████████| 1.9MB 6.4MB/s 
     |████████████████████████████████| 3.2MB 22.3MB/s 
     |████████████████████████████████| 890kB 39.2MB/s 
     |████████████████████████████████| 1.2MB 5.7MB/s 
     |████████████████████████████████| 51kB 3.0MB/s 


In [6]:
import transformers
transformers.__version__

'4.3.3'

### Preprocess

In [7]:
!wget -q --show-progress "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      


#### Preprocess twitter data

In [8]:
!cat data_twitter/train.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > data_twitter/train.txt.tmp
!cat data_twitter/test.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > data_twitter/test.txt.tmp
!cat data_twitter/dev.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > data_twitter/dev.txt.tmp

In [9]:
!mkdir processed_data_twitter

In [10]:
!python3 preprocess.py data_twitter/dev.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_twitter/dev.txt
!python3 preprocess.py data_twitter/train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_twitter/train.txt
!python3 preprocess.py data_twitter/test.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_twitter/test.txt

Downloading: 100% 434/434 [00:00<00:00, 403kB/s]
Downloading: 100% 1.22M/1.22M [00:00<00:00, 6.67MB/s]


#### Preprocess Peyma

In [11]:
with open('data_peyma/train.txt') as file:
  content = ''
  for line in file.readlines():
    content += line.replace(' ', '\u200c')
with open('data_peyma/train.txt.tmp', 'w') as file:
  file.write(content)

In [12]:
with open('data_peyma/test.txt') as file:
  content = ''
  for line in file.readlines():
    content += line.replace(' ', '\u200c')
with open('data_peyma/test.txt.tmp', 'w') as file:
  file.write(content)

In [13]:
with open('data_peyma/dev.txt') as file:
  content = ''
  for line in file.readlines():
    content += line.replace(' ', '\u200c')
with open('data_peyma/dev.txt.tmp', 'w') as file:
  file.write(content)

In [14]:
!cat data_peyma/train.txt.tmp | grep -v "^#" | cut -f 1,2 | tr '|' ' ' > data_peyma/train.txt.tmp2
!cat data_peyma/test.txt.tmp | grep -v "^#" | cut -f 1,2 | tr '|' ' ' > data_peyma/test.txt.tmp2
!cat data_peyma/dev.txt.tmp | grep -v "^#" | cut -f 1,2 | tr '|' ' ' > data_peyma/dev.txt.tmp2

In [15]:
!mkdir processed_data_peyma

In [16]:
!python3 preprocess.py data_peyma/dev.txt.tmp2 HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_peyma/dev.txt
!python3 preprocess.py data_peyma/train.txt.tmp2 HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_peyma/train.txt
!python3 preprocess.py data_peyma/test.txt.tmp2 HooshvareLab/bert-base-parsbert-uncased 256 > processed_data_peyma/test.txt

### Retrieve Labels

In [17]:
!cat processed_data_twitter/train.txt processed_data_twitter/test.txt processed_data_twitter/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels_twitter.txt

In [18]:
!cat processed_data_peyma/train.txt processed_data_peyma/test.txt processed_data_peyma/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels_peyma.txt

### Multi-task Training

In [ ]:
!python3 run_ner.py \
--data_dir_twitter ./processed_data_twitter/ \
--data_dir_peyma ./processed_data_peyma/ \
--labels_twitter labels_twitter.txt \
--labels_peyma labels_peyma.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir eval/ \
--max_seq_length_twitter  256 \
--max_seq_length_peyma  256 \
--per_device_train_batch_size 8 \
--save_steps 1000 \
--save_total_limit 1 \
--eval_steps 100 \
--logging_steps 100 \
--logging_first_step True \
--evaluation_strategy steps \
--seed 1 \
--num_train_epochs 5 \
--do_train \
--do_eval \
--do_predict

2021-03-09 09:10:43.254621: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
03/09/2021 09:10:44 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
03/09/2021 09:10:44 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=eval/, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=EvaluationStrategy.STEPS, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Mar09_09-10-44_8ece6402cd97, logging_first_step=True, logging_steps=100, save_steps=1000, save_total_limit=1, no